In [1]:
import urllib.request
import re
import mysql.connector as connection

In [2]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00448/carbon_nanotubes.csv"

In [3]:
file = urllib.request.urlopen(url)

In [4]:
mydb = connection.connect(host="localhost",user="root", passwd="mysql123", database = "testdb1", use_pure=True)

In [5]:
i = 0
for line in file:
    #print(line)
    
    if i == 0:
        header_columns = line.decode('utf-8')
        #print(header_columns)
        
        header_cols = header_columns.split(";")
        
        for i in range(len(header_cols)):
            header_cols[i] = header_cols[i].split("'")[0]
            
        for i in range(len(header_cols)):
            header_cols[i] = re.sub(" ", "_", header_cols[i])
            if re.findall("indice", header_cols[i]):
                header_cols[i] += " int(4)"
            else:
                header_cols[i] += " varchar(20)"
        
        create_query = "create table if not exists carbon_nanotubes(" + "%s ," * len(header_cols)

        create_query = re.split(",$", create_query)[0]

        create_query += ")"

        create_query = create_query % tuple(header_cols)

        cur = mydb.cursor()
        cur.execute(create_query)
        
        mydb.commit()
        
        i += 1
    
    else:
        
        line_decode = line.decode('utf-8')
        
        line_elements_list = line_decode.split(";")
        
        insert_val = ""
        for i in line_elements_list:
            if i.isnumeric():
                insert_val += i
            else:
                insert_val += "'"
                insert_val += i
                insert_val += "'"

            insert_val += ", "
        
        insert_val = re.split(", $", insert_val)[0]
        
        cur = mydb.cursor()
        cur.execute("insert into carbon_nanotubes values ( %s )" %(insert_val))

mydb.commit()

In [6]:
cur = mydb.cursor()
cur.execute("select count(*) from carbon_nanotubes")
cur.fetchall()

[(10721,)]